## Presentation Types with Hidden Types
## Batch Output to CSV in Sets

In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [ ]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0029.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [38]:
piece_list = ['CRIM_Model_0017', 
              "CRIM_Mass_0015_1", 
              "CRIM_Mass_0015_2", 
              "CRIM_Mass_0015_3", 
              "CRIM_Mass_0015_4", 
              "CRIM_Mass_0015_5"]

In [41]:

git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
mei = '.mei'
include_hidden_types = False
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 0
final = pd.DataFrame()

for piece in piece_list:
    file_url = git_prefix + piece 
    url = git_prefix + piece + mei
    piece = importScore(url)   
    nr = piece.getNoteRest(combineUnisons=combine_unisons)
    dur = piece.getDuration(df=nr)
    mel = piece.getMelodic(df=nr, kind='d', end=False)
    mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
    entries = piece.getEntries(mel_ng)

    output = classify_entries_as_presentation_types(piece, nr, mel_ng, entries, edit_distance_threshold, include_hidden_types)
    
    final = final.append(output, ignore_index=True)
    final['Validation'] = ""
    final['Comments'] = ""
# final.to_csv("corpus_classified.csv")
final.to_csv("saved_csv/CRIM_Mass_0015_P_Types.csv")

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [42]:
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Validation,Comments
0,Josquin Des Prés,Benedicta es,0.0,"[1/1.0, 9/1.0]",[P-8],"[0.0, 64.0]","[(2, 2, 2, -4)]",[64.0],"[[Contratenor], [Bassus]]",FUGA,2,False,,
1,Josquin Des Prés,Benedicta es,80.0,"[11/1.0, 15/1.0]",[P-8],"[80.0, 112.0]","[(-2, 4, -2, -2)]",[32.0],"[[Discantus], [Tenor]]",FUGA,2,False,,
2,Josquin Des Prés,Benedicta es,84.0,"[11/3.0, 15/3.0, 19/1.0, 23/1.0]","[P1, P12, P-8]","[84.0, 116.0, 144.0, 176.0]","[(2, -3, -2, -2)]","[32.0, 28.0, 32.0]","[[SextaVox], [Bassus], [Discantus], [Tenor]]",ID,4,False,,
3,Josquin Des Prés,Benedicta es,148.0,"[19/3.0, 23/3.0]",[P1],"[148.0, 180.0]","[(2, -3, 2, -2)]",[32.0],"[[SextaVox], [Bassus]]",FUGA,2,False,,
4,Josquin Des Prés,Benedicta es,154.0,"[20/2.0, 24/2.0]",[P1],"[154.0, 186.0]","[(4, -3, -3, -2)]",[32.0],"[[QuintaVox], [QuintaVox]]",FUGA,2,False,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,624.0,"[72/1.0, 74/1.0, 75/1.0, 76/1.0, 80/1.0, 81/1.0]","[P-8, P4, P-5, P5, P-5, P5]","[624.0, 652.0, 664.0, 676.0, 716.0, 716.0, 736.0]","[(-2, 2, 2, -3)]","[28.0, 12.0, 12.0, 40.0, 0.0, 20.0]","[[Discantus], [Tenor], [Contratenor 2], [Bassu...",FUGA,7,False,,
134,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,756.0,"[83/1.0, 86/1.0, 88/1.0, 90/1.0]","[P1, P1, P1]","[756.0, 780.0, 800.0, 824.0]","[(-3, 2, 2, -5)]","[24.0, 20.0, 24.0]","[[Bassus 2], [Bassus 1], [Bassus 2], [Bassus 1]]",ID,4,False,,
135,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,792.0,"[87/1.0, 89/1.0, 91/1.0]","[P-8, P8]","[792.0, 812.0, 840.0]","[(-3, 2, 2, -3)]","[20.0, 28.0]","[[Discantus], [Tenor], [Discantus]]",FUGA,3,False,,
136,Giovanni Pierluigi da Palestrina,Missa Benedicta es: Agnus Dei,796.0,"[87/1.0, 90/1.0, 91/1.0]","[P1, P1]","[796.0, 828.0, 844.0]","[(2, -2, -3, 4)]","[32.0, 16.0]","[[Contratenor 2], [Contratenor 1], [Contrateno...",FUGA,3,False,,
